In [35]:
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# we have a 6x9 grid of 54 squares
# we place a 1 layer thick wall around the outside
# each square has 4 values, up, down, left, right
# each square 

# Q values for each square, up, down, left, right for each square
maze = np.zeros((6, 9, 4))

# expected Q values for each square
maze_values = np.zeros((6, 9))

# set the reward for the goal
maze_values[1, 3] = 1

# set the wall
maze_values[3, 0:8] = -100

In [37]:
print(maze_values)

[[   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    1.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100. -100. -100. -100. -100. -100. -100. -100.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]]


In [38]:
def move(x, y, a, maze, maze_values):
    """ Gives the expected Q value of the next position for moving from the current position by taking action a.
    If the action is invalid, the current position is returned with a reward of -100.
    Actions are 0 = up, 1 = down, 2 = left, 3 = right.

    Args:
        x (_type_): the x coordinate of the current position
        y (_type_): the y coordinate of the current position
        a (_type_): the action to take, 0 = up, 1 = down, 2 = left, 3 = right
        maze (_type_): the Q values for each square
        maze_values (_type_): the expected Q values for each square
    """
    
    if a == 0:
        # up
        if x == 0:
            # can't move up
            return x, y, -100
        else:
            return x - 1, y, maze_values[x - 1, y] + maze[x, y, 0]
    elif a == 1:
        # down
        if x == 5:
            # can't move down
            return x, y, -100
        else:
            return x + 1, y, maze_values[x + 1, y]
    elif a == 2:
        # left
        if y == 0:
            # can't move left
            return x, y, -100
        else:
            return x, y - 1, maze_values[x, y - 1]
    elif a == 3:
        # right
        if y == 8:
            # can't move right
            return x, y, -100
        else:
            return x, y + 1, maze_values[x, y + 1]
    else:
        print("Invalid action, the action a must be 0(up), 1(down), 2(left), or 3(right)")
        raise ValueError("Invalid action")

In [39]:
def update_Q(x, y, a, maze, maze_values, learning_rate, discount_factor):
    """ Updates the Q value for the current position and action.

    Args:
        x (_type_): the x coordinate of the current position
        y (_type_): the y coordinate of the current position
        a (_type_): the action to take, 0 = up, 1 = down, 2 = left, 3 = right
        maze (_type_): the Q values for each square
        maze_values (_type_): the expected Q values for each square
        learning_rate (_type_): the learning rate
        discount_factor (_type_): the discount factor
    """
    # get the next position
    x_next, y_next, reward = move(x, y, a, maze, maze_values)

    # update the Q value
    maze[x, y, a] = (1 - learning_rate) * maze[x, y, a] + learning_rate * (reward + discount_factor * np.max(maze[x_next, y_next, :]))

In [40]:
# set the learning rate
learning_rate = 0.1

# set the discount factor
discount_factor = 0.9

# update the Q values for 100 iterations

for i in range(100):
    # update the Q values for each square
    for x in range(6):
        for y in range(9):
            for a in range(4):
                update_Q(x, y, a, maze, maze_values, learning_rate, discount_factor)

# set a maze which has the optimum action for each square
maze_optimum = np.zeros((6, 9))

# set the optimum action for each square
for x in range(6):
    for y in range(9):
        maze_optimum[x, y] = np.argmax(maze[x, y, :])

print(f"0: up, 1: down, 2: left, 3: right")
print(maze_optimum)
print("\n ------------------------------------ \n")
print(maze_values)



0: up, 1: down, 2: left, 3: right
[[1. 1. 1. 1. 2. 2. 2. 2. 2.]
 [3. 3. 3. 0. 2. 2. 2. 2. 2.]
 [3. 0. 0. 0. 2. 2. 2. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 3. 3. 3. 3. 3. 3. 3. 0.]
 [3. 0. 0. 0. 3. 0. 0. 3. 0.]]

 ------------------------------------ 

[[   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    1.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [-100. -100. -100. -100. -100. -100. -100. -100.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.]]
